In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

In [165]:
for idx in range(141, 151):
    with open(f'./session/{idx}.html', 'r', encoding='utf-8') as f:
        content = f.read()
    soup = BeautifulSoup(content, 'html.parser')
    # session
    session = int(soup.find('div', {'class', 'game_info'}).text[-3:])
session

150

In [2]:
Dataset = {
    'session': [],
    'team': [],
    'H': [],
    'AB': [],
    'BA': [],
    'earned_runs': [],
    'innings_pitched': [],
    'ERA': []
}
for idx in tqdm(range(1, 301)):
    with open(f'./session/{idx}.html', 'r', encoding='utf-8') as f:
        content = f.read()
    soup = BeautifulSoup(content, 'html.parser')
    # session
    session = int(soup.find('div', {'class', 'game_info'}).text[-3:])
    for _ in range(2):
        Dataset['session'].append(session)

    for i, div in enumerate(soup.find_all('div', {'class', 'RecordTable'}), start=1):
        # team_name
        if i in [1, 4]: 
            team = div.find('a').text
            Dataset['team'].append(team)
        
        # BA
        if i in [2, 5]: 
            data = list(div.find('tr', {'class', 'total'}).text.split(' '))
            AB, H = int(data[1]), int(data[3])
            BA = H / AB
            Dataset['AB'].append(AB)
            Dataset['H'].append(H)
            Dataset['BA'].append(BA)

        if i in [3, 6]:
            data = list(div.find('tr', {'class', 'total'}).text.split(' '))
            if '/' in data[1]:
                innings_pitched = int(data[1][:-3]) + (int(data[1][-3])/int(data[1][-1]))
                earned_runs = int(data[-5])
            else:
                innings_pitched, earned_runs = int(data[1]), int(data[-5])
            ERA = 9 * (earned_runs / innings_pitched)
            Dataset['innings_pitched'].append(innings_pitched)
            Dataset['earned_runs'].append(earned_runs)
            Dataset['ERA'].append(ERA)

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [00:31<00:00,  9.59it/s]


In [3]:
df = pd.DataFrame({
    'session': Dataset['session'],
    'team': Dataset['team'],
    'H': Dataset['H'],
    'AB': Dataset['AB'],
    'BA': Dataset['BA'],
    'earned_runs': Dataset['earned_runs'],
    'innings_pitched': Dataset['innings_pitched'],
    'ERA': Dataset['ERA']
})
df = df.replace('統一7-ELEVEn獅', '統一獅')
df

,session,team,H,AB,BA,earned_runs,innings_pitched,ERA
0,1,樂天桃猿,9,33,0.272727,1,9.000000,1.000000
1,1,中信兄弟,6,32,0.187500,4,9.000000,4.000000
2,2,味全龍,16,39,0.410256,5,9.000000,5.000000
3,2,中信兄弟,9,36,0.250000,7,9.000000,7.000000
4,3,富邦悍將,11,45,0.244444,3,12.000000,2.250000
...,...,...,...,...,...,...,...,...
595,298,味全龍,11,38,0.289474,6,10.000000,5.400000
596,299,統一獅,10,34,0.294118,7,8.666667,7.269231
597,299,味全龍,16,37,0.432432,6,9.000000,6.000000
598,300,富邦悍將,5,32,0.156250,4,8.000000,4.500000


In [4]:
df.to_csv('./session.csv', index=False)

In [255]:
df.iloc[np.argmax(df.BA)]

session                 108
team                   富邦悍將
H                        20
AB                       43
BA                 0.465116
earned_runs               7
innings_pitched         9.0
ERA                     7.0
Name: 214, dtype: object